# Step 3: Perform fit

In [1]:
import pycompwa.ui as pwa

2021-10-22 22:33:31,067 INFO [default] Logging to file disabled!
2021-10-22 22:33:31,067 [INFO] Log level: INFO
2021-10-22 22:33:31,067 [INFO] Current date and time: Fri Oct 22 22:33:31 2021



/opt/hostedtoolcache/Python/3.7.12/x64/lib/python3.7/site-packages/traitlets/traitlets.py:3050: FutureWarning: --rc={'figure.dpi': 96} for dict-traits is deprecated in traitlets 5.0. You can pass --rc <key=value> ... multiple times to add items to a dict.
  FutureWarning,


In [2]:
particle_list = pwa.read_particles("model.xml")
kinematics = pwa.create_helicity_kinematics("model.xml", particle_list)
kinematics.create_all_subsystems()
data_sample = pwa.read_root_data(input_file="generated_data.root")
phsp_sample = pwa.read_root_data(input_file="generated_phsp.root")
intensity_builder = pwa.IntensityBuilderXML(
    "model.xml", particle_list, kinematics, phsp_sample
)
intensity = intensity_builder.create_intensity()

2021-10-22 22:33:31,428 [INFO] HelicityKinematics::HelicityKinematics() | Initialized kinematics for reaction ( J/psi )->( gamma[ID=2] pi0[ID=3] pi0[ID=4] )
Event position to final state ID mapping:
0: 2
1: 3
2: 4



2021-10-22 22:33:31,429 [INFO] creating all Subsystems!


ERROR in cling::CIFactory::createCI(): cannot extract standard library include paths!


Invoking:
  

LC_ALL=C x86_64-linux-gnu-g++-7  -O3 -DNDEBUG -xc++ -E -v /dev/null 2>&1 >/dev/null | awk '/^#include </,/^End of search/{if (!/^#include </ && !/^End of search/){ print }}' | GREP_OPTIONS= grep -E "(c|g)\+\+"

Results was:


With exit code 

256

2021-10-22 22:33:31,858 [INFO] Setting phase space sample weights...


2021-10-22 22:33:31,868 [INFO] Updating data container content...


## 3.1 Define estimator

To perform a fit, you need to define an **estimator**. An estimator is a
measure for the discrepancy between the intensity model and the data
distribution to which you fit. To create an estimator we therefore generally
need:

In [3]:
data_set = kinematics.convert(data_sample)

Now we can create an estimator. In this example, we use an
**unbinned log likelihood** estimator. This estimator is used most commonly
in high-energy physics.

In [4]:
(
    estimator,
    initial_parameters,
) = pwa.create_unbinned_log_likelihood_function_tree_estimator(
    intensity, data_set
)

Notice that you not only receive a estimator object, but also a list of fit
parameters. You use this list of fit parameters to initialize the optimizer
later on.

The parameters contains the following info:

- initial value
- whether they are free or fixed
- optional upper and lower limits
- errors that give certain optimizers hints on the step size

In [5]:
initial_parameters

[strength_incoherent: 1 (fixed),
 mass_f21270: 1.2751 +- 0.0012 (fixed),
 Mass_gamma: 0 (fixed),
 Mass_jpsi: 3.0969 (fixed),
 Mass_neutralPion: 0.134977 +- 6e-06 (fixed),
 width_f21270: 0.185 (fixed),
 Radius_f21270: 1 (fixed),
 Magnitude_J/psi_to_f2(1270)_0+gamma_-1;f2(1270)_to_pi0_0+pi0_0;: 1,
 Phase_J/psi_to_f2(1270)_0+gamma_-1;f2(1270)_to_pi0_0+pi0_0;: 0,
 mass_f21950: 1.944 +- 0.012 (fixed),
 width_f21950: 0.472 (fixed),
 Radius_f21950: 1 (fixed),
 Magnitude_J/psi_to_f2(1950)_0+gamma_-1;f2(1950)_to_pi0_0+pi0_0;: 1,
 Phase_J/psi_to_f2(1950)_0+gamma_-1;f2(1950)_to_pi0_0+pi0_0;: 0,
 Magnitude_J/psi_to_f2(1270)_-1+gamma_-1;f2(1270)_to_pi0_0+pi0_0;: 1,
 Phase_J/psi_to_f2(1270)_-1+gamma_-1;f2(1270)_to_pi0_0+pi0_0;: 0,
 Magnitude_J/psi_to_f2(1950)_-1+gamma_-1;f2(1950)_to_pi0_0+pi0_0;: 1,
 Phase_J/psi_to_f2(1950)_-1+gamma_-1;f2(1950)_to_pi0_0+pi0_0;: 0,
 Magnitude_J/psi_to_f2(1270)_-2+gamma_-1;f2(1270)_to_pi0_0+pi0_0;: 1,
 Phase_J/psi_to_f2(1270)_-2+gamma_-1;f2(1270)_to_pi0_0+pi0_0;: 0,
 

## 3.2 Modify parameters

The fit parameters in this list have been initialized with the values stated
in the XML file or with default values if unspecified, but they can easily be
modified in Python as well:

In [6]:
print("\nthis parameter is initially not fixed:")
print(initial_parameters[8])

initial_parameters[7].is_fixed = True
initial_parameters[8].is_fixed = True
print("\nand now it is fixed:")
print(initial_parameters[8])


this parameter is initially not fixed:
Phase_J/psi_to_f2(1270)_0+gamma_-1;f2(1270)_to_pi0_0+pi0_0;: 0

and now it is fixed:
Phase_J/psi_to_f2(1270)_0+gamma_-1;f2(1270)_to_pi0_0+pi0_0;: 0 (fixed)


To make the fit process a bit more interesting, we modify one of the
parameters to a different initial value than the true value:

In [7]:
original_value = initial_parameters[12].value
initial_parameters[12]

Magnitude_J/psi_to_f2(1950)_0+gamma_-1;f2(1950)_to_pi0_0+pi0_0;: 1

In [8]:
initial_parameters[12].value = 2.0
initial_parameters[12]

Magnitude_J/psi_to_f2(1950)_0+gamma_-1;f2(1950)_to_pi0_0+pi0_0;: 2

## 3.3 Optimize the intensity model

The computation time depends on the complexity of the model, number of data
events, the size of the phase space sample, and the number of free
parameters.

In [9]:
print("data size", len(data_sample.events))
print("phsp size", len(phsp_sample.events))
print(
    "free parameters:",
    len([par for par in initial_parameters if not par.is_fixed]),
)

data size 10000
phsp size 100000
free parameters: 14


In [10]:
optimizer = pwa.MinuitIF()
fit_result = optimizer.optimize(estimator, initial_parameters)

2021-10-22 22:33:34,562 [INFO] MinuitIF::optimize() | Number of parameters (free): 32 (14)


2021-10-22 22:33:34,562 [INFO] Minuit2 strategy: medium
       GradientNCycles: 3
       GradientStepTolerance: 0.3
       GradientTolerance: 0.05
       HessianNCycles: 5
       HessianGradientNCycles: 2
       HessianStepTolerance: 0.3
       HessianG2Tolerance: 0.05


2021-10-22 22:33:34,563 [INFO] MinuitIF::optimize() | Starting migrad: maxCalls=0 tolerance=0.1


2021-10-22 22:33:50,166 [INFO] MinuitIF::optimize() | Migrad finished! Minimum is valid = 1


2021-10-22 22:33:50,169 [INFO] MinuitIF::optimize() | Starting hesse


2021-10-22 22:33:53,146 [INFO] MinuitIF::optimize() | Hesse finished


2021-10-22 22:33:53,147 [INFO] MinuitIF::optimize() | Minimization finished! LH = -5496.086088


In [11]:
fit_result.fit_duration_in_seconds

18

Let's check whether the fit parameters are close to the true values.

In [12]:
value = fit_result.final_parameters[12].value
error = fit_result.final_parameters[12].error
if abs(value - original_value) < 3 * error[0]:
    print("This value is close to", original_value, "again:")
    print(value, "+-", error[0])  # upper and lower error are equal in hesse
else:
    raise Exception("Value does not lie within 3 sigma!")

This value is close to 1.0 again:
0.9747981229026819 +- 0.10721047595228271


In [13]:
intensity.updateParametersFrom(fit_result.final_parameters)

## 3.4 Exporting and importing

Optimizing an intensity model can take a long time, so it is important that
you store the fit result in the end. We again write to an XML file (but note
that this has a different format than that of the amplitude model).

In [14]:
fit_result.write("fit_result.xml")

In [15]:
imported_fit_result = pwa.load("fit_result.xml")

exported_value = fit_result.final_parameters[12].value
imported_value = imported_fit_result.final_parameters[12].value

if exported_value != imported_value:
    raise Exception
else:
    print(imported_fit_result.final_parameters[12])

Magnitude_J/psi_to_f2(1950)_0+gamma_-1;f2(1950)_to_pi0_0+pi0_0;: 0.974798 +- 0.10721
